Mapping particle tracks from Ocean Parcels unto the Salish Sea Atlantis Boxes. 
Original code written by Bec Gordon & Javier Porobic, CSIRO.
Link to the [SSAM Ocean Parcels Repo](https://bitbucket.csiro.au/users/por07g/repos/ssam_oceanparcels/browse)

In [14]:
import os
import xarray as xr
import numpy as np
import geopandas as gpd
import pandas as pd
from netCDF4 import Dataset
from shapely.geometry import Point

In [15]:
shapefile_name = "/ocean/rlovindeer/Atlantis/ssam_oceanparcels/SalishSea/SalishSea_July172019_2/SalishSea_July172019.shp"
data_df = gpd.read_file(shapefile_name)
data_df = data_df.sort_values(by=['BOX_ID'])
box_depth = data_df['BOTZ']
box_area = data_df['AREA']
box_volume = box_area * box_depth
surface_volume = box_area * 25

#print(box_volume)

In [17]:
# Ocean Parcels Spill File
inputFileName = '5b_TurnPoint_Dilbit_2019-07-03_10day_forcing.nc'
num_particles = 2000

In [18]:
scenario = inputFileName.split(sep = '_')

In [19]:
# Oil type properties & spill location selection

Dilbit = {
    "Density": 1011.2, #kg/m^3
    "Naphthalene": 24, #mg/kg oil
    "Phenanthrene": 17,
    "Pyrene": 10,
    "Benzo": 3,
}

BunkerC = {
    "Density": 995.3,
    "Naphthalene": 680,
    "Phenanthrene": 796,
    "Pyrene": 266,
    "Benzo": 56,
}

Diesel = {
    "Density": 831.0,
    "Naphthalene": 3664,
    "Phenanthrene": 1000,
    "Pyrene": 0.000,
    "Benzo": 0.000,
}

Crude = {
    "Density": 884.7,
    "Naphthalene": 654,
    "Phenanthrene": 327,
    "Pyrene": 13,
    "Benzo": 2,
}

fuel_type = {
    "Dilbit" : Dilbit,
    "BunkerC" : BunkerC,
    "Diesel" : Diesel,
    "Crude" : Crude,
}

spill_volume = {
    "5b" : 2000, #m^3 
    "6a" : 15,
    "7a" : 1000,
    "4a" : 500,
}

spill_box_surface_volume = {
    "5b" : (322271112.331102*25), #m^3 
    "6a" : 15,
    "7a" : 1000,
    "4a" : 500,
}

In [21]:
# Calculations of oil per particle in mg/m^3/particle
release_start = scenario[3]
oil_per_particle = (fuel_type[scenario[2]]["Density"] * spill_volume[scenario[0]]) / num_particles
naph_per_particle = oil_per_particle * fuel_type[scenario[2]]["Naphthalene"] / spill_box_surface_volume[scenario[0]]
phen_per_particle = oil_per_particle * fuel_type[scenario[2]]["Phenanthrene"] / spill_box_surface_volume[scenario[0]]
pyrene_per_particle = oil_per_particle * fuel_type[scenario[2]]["Pyrene"] / spill_box_surface_volume[scenario[0]]
benzo_per_particle = oil_per_particle * fuel_type[scenario[2]]["Benzo"] / spill_box_surface_volume[scenario[0]]
release_start_time = np.datetime64(release_start)

In [38]:
mgoil_per_particle = oil_per_particle * 1e6 / spill_box_surface_volume[scenario[0]]

In [30]:
# Calculating the surface concentration of each contaminant for the entire spill
oil_full = oil_per_particle * num_particles * 1e6 / spill_box_surface_volume[scenario[0]]
naph_full = naph_per_particle * num_particles
phen_full = phen_per_particle * num_particles
pyrene_full = pyrene_per_particle * num_particles
benzo_full = benzo_per_particle * num_particles

print(str(scenario[2])+' spill concentration of '+str(oil_full)+' in mg/m^3')
print('Naphthalene spill concentration of '+str(naph_full)+' in mg/m^3')
print('Phenanthrene spill concentration of '+str(phen_full)+' in mg/m^3')
print('Pyrene spill concentration of '+str(pyrene_full)+' in mg/m^3')
print('Benzo spill concentration of '+str(benzo_full)+' in mg/m^3')

Dilbit spill concentration of 251.01846521349788 in mg/m^3
Naphthalene spill concentration of 0.00602444316512395 in mg/m^3
Phenanthrene spill concentration of 0.004267313908629464 in mg/m^3
Pyrene spill concentration of 0.0025101846521349788 in mg/m^3
Benzo spill concentration of 0.0007530553956404938 in mg/m^3


In [31]:
numLayers = 7
numSites = data_df.shape[0]
numTargetSites = numSites

#outputDT = 60*60
outputDT = 43100.00

stepsPerDay = int(86400.0/ outputDT)
numStepsPerDT = int(outputDT/3600.0)

debug = False

In [32]:
pfile = xr.open_dataset(str(inputFileName), decode_cf=True)

lon = np.ma.filled(pfile.variables['lon'], np.nan)
lat = np.ma.filled(pfile.variables['lat'], np.nan)
time = np.ma.filled(pfile.variables['time'], np.nan)
z = np.ma.filled(pfile.variables['z'], np.nan)
probs = np.ma.filled(pfile.variables['decay_value'], np.nan)

In [33]:
numParticles = lon.shape[0]
trackDates = []

for i in range(0,numParticles):
    #print(time[i][0])
    trackDates.append(time[i][0])

RDiff = max(trackDates) - min(trackDates)
minDate = np.datetime64(release_start+"T00:30:00")
ts = pd.to_datetime(str(minDate))
d = ts.strftime('%Y-%m-%d %H:%M:%S')
print(d)

2019-07-03 00:30:00


In [34]:
numReleaseDays = 1
numReleaseSteps = numReleaseDays * stepsPerDay

trackLength = len(lon[0])

print('trackLength = ' + str(trackLength))
print('numStepsPerDT = ' + str(numStepsPerDT))
numSteps = int(trackLength / numStepsPerDT)


trackLength = 21
numStepsPerDT = 11


In [35]:
# Create the netcdf output file

netcdfFileName = "SSAM_Scenario_" + scenario[0] + "_" + scenario[3] + "_" + str(num_particles) + ".nc"
try:
    os.remove(netcdfFileName)
except:
    pass
ncfile = Dataset(netcdfFileName, "w", format="NETCDF4", clobber=True)
Dataset.set_fill_on(ncfile)

# Dimensions
time = ncfile.createDimension("t", None)
b = ncfile.createDimension("b", numTargetSites)
z = ncfile.createDimension("z", numLayers)

In [36]:
# Variables
times = ncfile.createVariable("t",np.float64, ("t",))
oil = ncfile.createVariable("oil",np.float64,("t", "b"))
Naphthalene = ncfile.createVariable("Naphthalene",np.float64, ("t", "b", "z"))
Phenanthrene = ncfile.createVariable("Phenanthrene",np.float64,("t", "b", "z"))
Pyrene = ncfile.createVariable("Pyrene",np.float64,("t", "b", "z"))
Benzo = ncfile.createVariable("Benzo",np.float64,("t", "b", "z"))

# Attributes
Naphthalene.units = "mgPAH/m^3"
Naphthalene.long_name = "Naphthalene"
Naphthalene.missing_value = 0.0000
Naphthalene.valid_min = 0.0000
Naphthalene.valid_max = 100000000.0

Phenanthrene.units = "mgPAH/m^3"
Phenanthrene.long_name = "Phenanthrene"
Phenanthrene.missing_value = 0.0000
Phenanthrene.valid_min = 0.0000
Phenanthrene.valid_max = 100000000.0

Pyrene.units = "mgPAH/m^3"
Pyrene.long_name = "Pyrene"
Pyrene.missing_value = 0.0000
Pyrene.valid_min = 0.0000
Pyrene.valid_max = 100000000.0

Benzo.units = "mgPAH/m^3"
Benzo.long_name = "Benzo(a)pyrene"
Benzo.missing_value = 0.0000
Benzo.valid_min = 0.0000
Benzo.valid_max = 100000000.0

oil.units = "kgOil/m^3"
oil.long_name = "Oil"

times.units = "seconds since " + d
times.dt = outputDT
times.long_name = "time"

OPTION 1: TO CREATE FORCING FILE WITH ZEROES
Creates a contaminant forcing file with no contaminants

numSteps = int((70*60*60*24*365)/outputDT)
timeData = np.arange(0,(numSteps)*outputDT,outputDT)
times[:] = timeData

FillerData = np.zeros((numSteps, numTargetSites, numLayers))

Naphthalene[:,:,:] = FillerData
Phenanthrene[:,:,:] = FillerData
Pyrene[:,:,:] = FillerData
Benzo[:,:,:] = FillerData

ncfile.close()

In [37]:
# OPTION 2: CREATE CONTAMINANT FORCING FILE
# Populate variables with contaminant data

timeData = np.arange(0,(numSteps + numReleaseSteps)*outputDT,outputDT)
times[:] = timeData

Surface_particles = np.zeros((numSteps + numReleaseSteps, numTargetSites))
Dispersed_particles = np.zeros((numSteps + numReleaseSteps, numTargetSites, numLayers))

In [39]:
for partIndex in range(0, numParticles):

    trackDateDiff = trackDates[partIndex] - minDate
    trackDateDiff = trackDateDiff/ np.timedelta64(1, 's')

    timeOffset = int(abs((trackDateDiff /outputDT)))

    for stepIndex in range(0, numSteps):
        timeValue = stepIndex + timeOffset

        partLon = lon[partIndex][stepIndex * numStepsPerDT]
        partLat = lat[partIndex][stepIndex * numStepsPerDT]
        partProb = probs[partIndex][stepIndex * numStepsPerDT]

        matchFound = 0

        for targetIndex in range (0, numTargetSites):

            box_id = data_df.iloc[targetIndex].BOX_ID
            box_coordinates = data_df.iloc[targetIndex].geometry
            find_particle = box_coordinates.contains(Point(partLon, partLat))
            
            if box_depth[targetIndex] < 26:
                layer = 0
            elif box_depth[targetIndex] == 50:
                layer = 1
            elif box_depth[targetIndex] == 100:
                layer = 2
            elif box_depth[targetIndex] == 200:
                layer = 3
            elif box_depth[targetIndex] > 200 and box_depth[targetIndex] < 401:
                layer = 4
            elif box_depth[targetIndex] > 400:
                layer = 5

            if find_particle:
                Dispersed_particles[timeValue][box_id][layer] = Dispersed_particles[timeValue][box_id][layer] + partProb
                Surface_particles[timeValue][box_id] = Surface_particles[timeValue][box_id] + partProb
                
                # uncomment line below to ignore particle decay during debugging.
                # Dispersed_particles[timeValue][box_id] = Dispersed_particles[timeValue][box_id] + 1.0
            
                #matchFound = 1
                #if debug:
                #    print('At time ' + str(timeValue) + ' Particle (' + str(partIndex) + ') in box ' + str(data_df.iloc[targetIndex].BOX_ID))

                break

        if matchFound == 0:
            if debug:
                print('No match for particle')
                print(partLon, partLat)

        #break

oil[:,:] = Surface_particles * mgoil_per_particle
Naphthalene[:,:,:] = Dispersed_particles * naph_per_particle
Phenanthrene[:,:,:] = Dispersed_particles * phen_per_particle
Pyrene[:,:,:] = Dispersed_particles * pyrene_per_particle
Benzo[:,:,:] = Dispersed_particles * benzo_per_particle

ncfile.close()


In [40]:
np.histogram(Surface_particles)

(array([386,   1,   0,   0,   0,   1,   0,   0,   1,   1]),
 array([  0. ,  75.1, 150.2, 225.3, 300.4, 375.5, 450.6, 525.7, 600.8,
        675.9, 751. ]))